In [10]:
import pyodbc
import pandas as pd
import Resources.config as cf
from sklearn.model_selection import train_test_split

#Possible Model Types
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#Visualizations
import seaborn as sns
import matplotlib.pyplot as plt

server = 'weatherdata.database.windows.net' 
database = 'weather' 
username = cf.username
password = cf.password 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)


In [11]:
#Import data into dataframes

#city_attributes
query1 = "SELECT [City],[Country],[Latitude],[Longitude] from [weather].[dbo].[city_attributes];"
df_city_attributes = pd.read_sql(query1, cnxn)

#humidity
query2 = "select [datetime],Vancouver,Portland,San_Francisco,Seattle,Los_Angeles,San_Diego,Las_Vegas,Phoenix,Albuquerque,Denver,\
San_Antonio,Dallas,Houston,Kansas_City,Minneapolis,Saint_Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,\
Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New_York,Montreal,Boston,Beersheba,Tel_Aviv_District,Eilat,Haifa,Nahariyya,\
Jerusalem from [weather].[dbo].humidity;"
df_humidity = pd.read_sql(query2, cnxn)

#pressure
query3 = "select [datetime],Vancouver,Portland,San_Francisco,Seattle,Los_Angeles,San_Diego,Las_Vegas,Phoenix,Albuquerque,Denver,\
San_Antonio,Dallas,Houston,Kansas_City,Minneapolis,Saint_Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,\
Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New_York,Montreal,Boston,Beersheba,Tel_Aviv_District,Eilat,Haifa,Nahariyya,\
Jerusalem from [weather].[dbo].pressure;"
df_pressure = pd.read_sql(query3, cnxn)

#temperature
query4 = "select [datetime],Vancouver,Portland,San_Francisco,Seattle,Los_Angeles,San_Diego,Las_Vegas,Phoenix,Albuquerque,Denver,\
San_Antonio,Dallas,Houston,Kansas_City,Minneapolis,Saint_Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,\
Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New_York,Montreal,Boston,Beersheba,Tel_Aviv_District,Eilat,Haifa,Nahariyya,\
Jerusalem from [weather].[dbo].temperature;"
df_temperature = pd.read_sql(query4, cnxn)

#weather_description
query5 = "select [datetime],Vancouver,Portland,San_Francisco,Seattle,Los_Angeles,San_Diego,Las_Vegas,Phoenix,Albuquerque,Denver,\
San_Antonio,Dallas,Houston,Kansas_City,Minneapolis,Saint_Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,\
Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New_York,Montreal,Boston,Beersheba,Tel_Aviv_District,Eilat,Haifa,Nahariyya,\
Jerusalem from [weather].[dbo].weather_description;"
df_weather_description = pd.read_sql(query5, cnxn)

#wind_direction
query6 = "select [datetime],Vancouver,Portland,San_Francisco,Seattle,Los_Angeles,San_Diego,Las_Vegas,Phoenix,Albuquerque,Denver,\
San_Antonio,Dallas,Houston,Kansas_City,Minneapolis,Saint_Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,\
Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New_York,Montreal,Boston,Beersheba,Tel_Aviv_District,Eilat,Haifa,Nahariyya,\
Jerusalem from [weather].[dbo].wind_direction;"
df_wind_direction = pd.read_sql(query6, cnxn)

#wind_speed
query7 = "select [datetime],Vancouver,Portland,San_Francisco,Seattle,Los_Angeles,San_Diego,Las_Vegas,Phoenix,Albuquerque,Denver,\
San_Antonio,Dallas,Houston,Kansas_City,Minneapolis,Saint_Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,\
Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New_York,Montreal,Boston,Beersheba,Tel_Aviv_District,Eilat,Haifa,Nahariyya,\
Jerusalem from [weather].[dbo].wind_speed;"
df_wind_speed = pd.read_sql(query7, cnxn)

In [12]:
df_city_attributes.head()

,City,Country,Latitude,Longitude
0,Vancouver,Canada,49.249660,-123.119339
1,Portland,United States,45.523449,-122.676208
2,San Francisco,United States,37.774929,-122.419418
3,Seattle,United States,47.606209,-122.332069
4,Los Angeles,United States,34.052231,-118.243683


In [ ]:
#Data exploration, use matplotlib or seaborn

In [ ]:
#Data cleansing if needed, create functions

In [ ]:
#Train Test/Split
#x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)

In [14]:
#Choose Model Type, maybe for extra credit use neural network possibly Keras keras functional or sequential

# lr = LogisticRegression()
# rf = RandomForestClassifier(bootstrap=False)
# gbc = GradientBoostingClassifier()
# dt = DecisionTreeClassifier()
# svc = SVC()
# knn= KNeighborsClassifier()


'\nlr = LogisticRegression()\nrf = RandomForestClassifier(bootstrap=False)\ngbc = GradientBoostingClassifier()\ndt = DecisionTreeClassifier()\nsvc = SVC()\nknn= KNeighborsClassifier()\n'

In [ ]:
#Train Model
# lr.fit(x_train,y_train)
# rf.fit(x_train,y_train)
# gbc.fit(x_train,y_train)
# dt.fit(x_train,y_train)
# svc.fit(x_train, y_train)
# knn.fit(x_train, y_train)

In [ ]:
#Train Model


In [ ]:
#Check Accuracy and loss

In [ ]:
#Inference test data

In [ ]:
#Save Model and pickle the data

In [ ]:
#Export Data back to SQL Server